In [1]:
import pandas as pd


In [2]:
PATH = "../data/ibamco.csv"

In [3]:
df = pd.read_csv(PATH, dtype={"zipcode_iber": object, "zipcode_cac": object})
df["zipcode_iber"] = df["zipcode_iber"].str.zfill(5)
df["zipcode_cac"] = df["zipcode_cac"].str.zfill(5)

In [4]:
df[df.Origin.isnull()]

,media,area_of_influence,ojd,url,CCAA,PROV,AREA,platform,PGD,distribution,...,province_iber,CCAA_iber,type_media_cac,address_cac,lat_cac,lng_cac,zipcode_cac,classification_ojd,sub_classification_ojd,unique_visitors_ojd


In [5]:
import numpy as np
df['ID'] = np.arange(len(df))

In [6]:
# group all other columns, but keep the first id in grouping 

df = df[['ID', 'Origin', 'media', 'url', 'zipcode_iber', 'zipcode_cac']]
df['url'].fillna('', inplace=True)

In [7]:
df['combine'] = df['Origin'] + "~" + df['media'] + "~" + df['url'] 

In [8]:
df = df[['ID', 'combine', 'zipcode_iber', 'zipcode_cac']]
df

,ID,combine,zipcode_iber,zipcode_cac
0,0,"Iberfier, CAC, OJD~el 9 nou~el9nou.cat",08500,08500
1,1,"Iberfier, CAC, OJD~el 9 nou~el9nou.cat",08500,08500
2,2,"Iberfier, CAC, OJD~betevé~beteve.cat",08018,08099
3,3,"Iberfier, CAC, OJD~pirineus tv~pirineustv.cat",25700,25700
4,4,"Iberfier, CAC, OJD~ràdio sabadell~radiosabadel...",08202,08202
...,...,...,...,...
3691,3691,OJD~urgelltv~urgelltv.cat,NaN,NaN
3692,3692,OJD~21radio~21radio.cat,NaN,NaN
3693,3693,OJD~aue~aue.cat,NaN,NaN
3694,3694,OJD~fembase~fembase.cat,NaN,NaN


In [9]:
combine_uniques_ids = df[['combine', 'ID']].drop_duplicates(subset='combine', keep='first')
combine_uniques_ids['zipcode_iber'] = np.nan
combine_uniques_ids['zipcode_cac'] = np.nan

In [10]:
combine_uniques_ids

,combine,ID,zipcode_iber,zipcode_cac
0,"Iberfier, CAC, OJD~el 9 nou~el9nou.cat",0,NaN,NaN
2,"Iberfier, CAC, OJD~betevé~beteve.cat",2,NaN,NaN
3,"Iberfier, CAC, OJD~pirineus tv~pirineustv.cat",3,NaN,NaN
4,"Iberfier, CAC, OJD~ràdio sabadell~radiosabadel...",4,NaN,NaN
5,"AMIC, Iberfier, OJD~adolescents~adolescents.cat",5,NaN,NaN
...,...,...,...,...
3691,OJD~urgelltv~urgelltv.cat,3691,NaN,NaN
3692,OJD~21radio~21radio.cat,3692,NaN,NaN
3693,OJD~aue~aue.cat,3693,NaN,NaN
3694,OJD~fembase~fembase.cat,3694,NaN,NaN


In [11]:
combine_uniques = df['combine'].unique()
new_df = pd.DataFrame(combine_uniques, columns=['combine'])
combine_uniques

array(['Iberfier, CAC, OJD~el 9 nou~el9nou.cat',
       'Iberfier, CAC, OJD~betevé~beteve.cat',
       'Iberfier, CAC, OJD~pirineus tv~pirineustv.cat', ...,
       'OJD~aue~aue.cat', 'OJD~fembase~fembase.cat',
       'OJD~amicib~amicib.media'], dtype=object)

In [12]:
for index, row in combine_uniques_ids.iterrows():
    rows = df[df['combine'] == row['combine']]
    z_iber = rows['zipcode_iber'].dropna().unique()
    z_cac = rows['zipcode_cac'].dropna().unique()
    media = row['combine'].split('~')[1]
    combine_uniques_ids.loc[index, 'zipcode_iber'] = ",".join(z_iber)
    combine_uniques_ids.loc[index, 'zipcode_cac'] = ",".join(z_cac)

combine_uniques_ids


/tmp/ipykernel_20671/2217180133.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '08500' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  combine_uniques_ids.loc[index, 'zipcode_iber'] = ",".join(z_iber)
/tmp/ipykernel_20671/2217180133.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '08500' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  combine_uniques_ids.loc[index, 'zipcode_cac'] = ",".join(z_cac)


,combine,ID,zipcode_iber,zipcode_cac
0,"Iberfier, CAC, OJD~el 9 nou~el9nou.cat",0,08500,08500
2,"Iberfier, CAC, OJD~betevé~beteve.cat",2,08018,08099
3,"Iberfier, CAC, OJD~pirineus tv~pirineustv.cat",3,25700,25700
4,"Iberfier, CAC, OJD~ràdio sabadell~radiosabadel...",4,08202,08202
5,"AMIC, Iberfier, OJD~adolescents~adolescents.cat",5,08500,
...,...,...,...,...
3691,OJD~urgelltv~urgelltv.cat,3691,,
3692,OJD~21radio~21radio.cat,3692,,
3693,OJD~aue~aue.cat,3693,,
3694,OJD~fembase~fembase.cat,3694,,


In [13]:
df.sort_values(by='ID')

,ID,combine,zipcode_iber,zipcode_cac
0,0,"Iberfier, CAC, OJD~el 9 nou~el9nou.cat",08500,08500
1,1,"Iberfier, CAC, OJD~el 9 nou~el9nou.cat",08500,08500
2,2,"Iberfier, CAC, OJD~betevé~beteve.cat",08018,08099
3,3,"Iberfier, CAC, OJD~pirineus tv~pirineustv.cat",25700,25700
4,4,"Iberfier, CAC, OJD~ràdio sabadell~radiosabadel...",08202,08202
...,...,...,...,...
3691,3691,OJD~urgelltv~urgelltv.cat,NaN,NaN
3692,3692,OJD~21radio~21radio.cat,NaN,NaN
3693,3693,OJD~aue~aue.cat,NaN,NaN
3694,3694,OJD~fembase~fembase.cat,NaN,NaN


In [14]:
combine_uniques_ids.sort_values(by='ID')

,combine,ID,zipcode_iber,zipcode_cac
0,"Iberfier, CAC, OJD~el 9 nou~el9nou.cat",0,08500,08500
2,"Iberfier, CAC, OJD~betevé~beteve.cat",2,08018,08099
3,"Iberfier, CAC, OJD~pirineus tv~pirineustv.cat",3,25700,25700
4,"Iberfier, CAC, OJD~ràdio sabadell~radiosabadel...",4,08202,08202
5,"AMIC, Iberfier, OJD~adolescents~adolescents.cat",5,08500,
...,...,...,...,...
3691,OJD~urgelltv~urgelltv.cat,3691,,
3692,OJD~21radio~21radio.cat,3692,,
3693,OJD~aue~aue.cat,3693,,
3694,OJD~fembase~fembase.cat,3694,,


In [15]:
combine_uniques_ids['Origin'] = combine_uniques_ids['combine'].str.split('~').str[0]
combine_uniques_ids['media'] = combine_uniques_ids['combine'].str.split('~').str[1]
combine_uniques_ids['url'] = combine_uniques_ids['combine'].str.split('~').str[2]
combine_uniques_ids


,combine,ID,zipcode_iber,zipcode_cac,Origin,media,url
0,"Iberfier, CAC, OJD~el 9 nou~el9nou.cat",0,08500,08500,"Iberfier, CAC, OJD",el 9 nou,el9nou.cat
2,"Iberfier, CAC, OJD~betevé~beteve.cat",2,08018,08099,"Iberfier, CAC, OJD",betevé,beteve.cat
3,"Iberfier, CAC, OJD~pirineus tv~pirineustv.cat",3,25700,25700,"Iberfier, CAC, OJD",pirineus tv,pirineustv.cat
4,"Iberfier, CAC, OJD~ràdio sabadell~radiosabadel...",4,08202,08202,"Iberfier, CAC, OJD",ràdio sabadell,radiosabadell.fm
5,"AMIC, Iberfier, OJD~adolescents~adolescents.cat",5,08500,,"AMIC, Iberfier, OJD",adolescents,adolescents.cat
...,...,...,...,...,...,...,...
3691,OJD~urgelltv~urgelltv.cat,3691,,,OJD,urgelltv,urgelltv.cat
3692,OJD~21radio~21radio.cat,3692,,,OJD,21radio,21radio.cat
3693,OJD~aue~aue.cat,3693,,,OJD,aue,aue.cat
3694,OJD~fembase~fembase.cat,3694,,,OJD,fembase,fembase.cat


In [16]:
combine_uniques_ids[['ID','Origin', 'media', 'url', 'zipcode_iber', 'zipcode_cac']].sort_values('ID').to_excel('ibamco_sheet.xlsx', 'main', index=False)

In [33]:
df[df.Origin.isnull()]

,ID,combine,zipcode_iber,zipcode_cac,Origin,media,url
186,186,NaN,NaN,25140,NaN,NaN,NaN
187,187,NaN,NaN,43786,NaN,NaN,NaN
188,188,NaN,NaN,25250,NaN,NaN,NaN
189,189,NaN,NaN,17867,NaN,NaN,NaN
190,190,NaN,NaN,08261,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2466,2466,NaN,NaN,17489,NaN,NaN,NaN
2467,2467,NaN,NaN,08589,NaN,NaN,NaN
2468,2468,NaN,NaN,25740,NaN,NaN,NaN
2469,2469,NaN,NaN,25670,NaN,NaN,NaN
